#### Libraries

In [6]:
import pickle as pkl
import networkx as nx
import numpy as np
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

with open("training_data.pkl", 'rb') as file:
    train_graphs = pkl.load(file)


#### Direct Product Kernel

In [113]:
import scipy.sparse as ss
def DPKernel(adjacencymatrix):
    rho = ss.linalg.norm(adjacencymatrix)
    infsum = ss.linalg.inv( ss.eye(adjacencymatrix.shape[0], format='csc') - rho*adjacencymatrix )
    k = np.sum(np.abs(infsum))
    return k

Calculate DP Kernel

In [ ]:
N = len(train_graphs)
K = np.zeros((N,N))
for i in tqdm(range(N)):
    G1 = train_graphs[i]
    for j in range(i+1): #K will be symmetric, so only calculate upper and then change
        G2 = train_graphs[j]        
        Gx = nx.tensor_product(G1, G2)
        Ax = nx.adjacency_matrix(Gx)
        K[i,j] = DPKernel(Ax)


K = K + K.T - np.diag(np.diag(K)) #extract diagonal and then make into diagonal matrix

Save data

In [118]:
np.savetxt('DPKernel.txt', K)

### Scraps


In [91]:
Gx = nx.tensor_product(train_graphs[1], train_graphs[2])
#Adjacency
Ax = nx.adjacency_matrix(Gx)  # ss.csr_matrix.toarray(nx.adjacency_matrix(G))
DPKernel(Ax)

428.68649496070907

In [ ]:
from lib import draw
import matplotlib.pyplot as plt
G0 = train_graphs[0].subgraph([4, 1, 7, 0, 15, 8])
G1 = train_graphs[1].subgraph( [0, 1, 2, 3, 5, 6, 7, 8, 10, 11])
plt.figure(figsize=(12, 8))
plt.subplot(1,2,1)
draw(G0)
plt.subplot(1,2,2)
draw(G1)


In [ ]:
Gx = nx.tensor_product(G0, G1)
pos = nx.spring_layout(Gx, seed=1, k=0.3)
nx.draw(Gx, pos)
node_labels = nx.get_node_attributes(Gx, "labels")
nx.draw_networkx_labels(Gx, pos, node_labels, font_size=8);
# edge_labels = nx.get_edge_attributes(P, "labels")
# nx.draw_networkx_labels(P, pos, edge_labels, font_size=8);

In [ ]:
# plt.subplot(1,1,1)
# draw(P)
Gx.nodes()